In [4]:
#FONCTIONS A AJOUTER

# Recuperation des arcs
function getEdge(kep)
    edge_list = Array{Array{Int64,1},1}()
    for (ind,arc) in enumerate(edges(kep))
            push!(edge_list,[src(arc), dst(arc)])
    end
    return edge_list
end

getEdge(pool)

#Recuperation de tous les chemins de longueur L+1
function getPath_L(pool)
    all_path = simplecycles_hawick_james(pool) #Recuperation de tous les chemins du graphe
    path_L=[]
    L=6
    for i in all_path #Recuperation des chemins de longueur L+1 dans path_L
        if length(i) == L+1
            push!(path_L, i)
        end
    end
    return path_L
end

getEdge (generic function with 1 method)

In [5]:
# Le probleme

# Le modèle : choisir un des 2 selon votre ORDI
#model = Model()

#model=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
set_optimizer_attribute(model, "OutputFlag", 0) #???? parametre a modifier

# Le solver
statut = set_optimizer(model, GLPK.Optimizer)

# Les donnees
path_L = getPath_L(pool)

#Ensemble des arcs sous la forme [i,j]
edge_list = getEdge(pool); #Penser a ajouter la fonction PROBLEME_SANS_REFORMULATION

#Nombre de sommets
E = ne(pool)
V = nv(pool)

#Les variables
@variable(model, x[1:E,, 1:E], Edge)

#La fonction objective
@objective(model, Max, sum(x[e[1],e[2]] for e=1:E))

#les contraintes
#Premiere contrainte
for i = 1:V
    E_1 = [] #Ensemble des arcs ou i est le depart
    E_2 = [] #Ensemble des arcs ou i est l arrivee
    #Recuperer les arcs ou il y a i en extremite
    for e = edge_list
        if e[1] == i
            push!(E_1, e[2])
        end
        if e[2] == i
            push!(E_2, e[1])
        end
    end     
    #La contrainte sur le sommet i
    @constraint(model, sum(x[e, i] for e=E_2) == sum(x[i, e for e=E_1))
end

#Deuxieme contrainte
for i = 1:V
    E_i = [] #Ensemble des arcs ou i est l arrivee
    #Recuperer les arcs ou il y a i en extremite
    for e=1:E
        if e[1] == i
            push!(E_i, e[2])
        end
    end
    #La contrainte sur le sommet i
    @constraint(model, sum(x[i, e[2]] for e =1:E_i) <= 1)
end

#Troisieme contrainte
for p in path_L #Pour tous les chemins de longueur L+1
    @constraint(model, sum(x[p[j], p[j+1]] for j=1:L) <= L-1)

LoadError: UndefVarError: set_optimizer not defined